# Order DigiKey Components From BOM 
Python 3.10.16

This script generates a CSV BOM to upload to [DigiKey myLists](https://www.digikey.ca/en/mylists/): `BIODYN-100 vXX BOM.myLists.csv`.

In [49]:
%pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
# Read Tidy BOM
import pandas as pd
bom_file_name = "data/BIODYN-100 v58 BOM Tidy.csv"

df = pd.read_csv(bom_file_name, sep=";")
df.head(2)

,Part,Value,Device,Footprint Name,Detailed Description,Link,Manuf Part Number,Datasheet URL,Description,Manufacturer,DigiKey Part Number,Pricing,Stock
0,EMG1:D1,1N4148DO35-10,1N4148DO35-10,DO35-10,DIODE,https://www.digikey.ca/en/products/detail/onse...,1N4148,https://www.mccsemi.com/pdf/Products/1N4148WX(...,DIODE STANDARD 75V 150MA SOD323,MCC (Micro Commercial Components),1N4148WXTPMSCT-ND,"[{'break_quantity': 1, 'total_price': 0.1, 'un...",1145155.0
1,EMG1:D2,1N4148DO35-10,1N4148DO35-10,DO35-10,DIODE,https://www.digikey.ca/en/products/detail/onse...,1N4148,https://www.mccsemi.com/pdf/Products/1N4148WX(...,DIODE STANDARD 75V 150MA SOD323,MCC (Micro Commercial Components),1N4148WXTPMSCT-ND,"[{'break_quantity': 1, 'total_price': 0.1, 'un...",1145155.0


In [51]:
outfile = "BIODYN-100 v58 BOM.myLists.csv"

df['Count'] = 1
agg_funcs = {
	'Count': 'sum', 
	"Manufacturer": 'first',
	"Manuf Part Number": 'first',
	'DigiKey Part Number': 'first', 
	}
df_by_part = df.groupby(df['Manuf Part Number']).aggregate(agg_funcs)
df_by_part.rename(columns={'Count': 'Quantity', 
							 "Manuf Part Number": "Manufacturer Part Number",
							 "Manufacturer": "Manufacturer Name",
							 "DigiKey Part Number": "Requested Part Number"}, 
							 inplace=True)
df_by_part['Index'] = range(1, len(df_by_part) + 1)
df_by_part.set_index('Index', inplace=True)
new_order = ['Manufacturer Part Number', 'Manufacturer Name', 'Requested Part Number', 'Quantity'];
df_by_part = df_by_part[new_order]

df_by_part.head(5)

,Manufacturer Part Number,Manufacturer Name,Requested Part Number,Quantity
Index,,,,
1,1N4148,MCC (Micro Commercial Components),1N4148WXTPMSCT-ND,2
2,885012005024,Würth Elektronik,732-7440-1-ND,5
3,BQ21040DBVR,Texas Instruments,296-47315-1-ND,1
4,BQ29700DSER,Texas Instruments,296-43985-1-ND,1
5,C0402C150J8HACTU,KEMET,399-C0402C150J8HACTUCT-ND,2


In [52]:
df_by_part.to_csv(outfile, index=True)


Copyright © 2025 Callum Mackenzie 